In [ ]:
{
   "cells": [
      {
         "cell_type": "markdown",
         "metadata": {
            "id": "Yx76qjaGKwIS"
         },
         "source": [
            "# 既存資料からチャットボットを作成する(Retrieval Augumented Generation:RAG)\n",
            "\n",
            "- 参考\n",
            "  - https://colab.research.google.com/github/nyanta012/demo/blob/main/sentence_retrieval.ipynb\n",
            "  - https://python.langchain.com/en/latest/modules/chains/index_examples/vector_db_qa.html\n",
            "  - https://note.com/mahlab/n/nb6677d0fc7c2\n",
            "- Chromadbがすごい\n",
            "  - https://www.trychroma.com/\n",
            "  - https://github.com/chroma-core/chroma\n",
            "- ChatGPTのSurvey論文\n",
            "  - https://arxiv.org/pdf/2304.01852.pdf\n",
            "  \n"
         ]
      },
      {
         "cell_type": "code",
         "execution_count": 1,
         "metadata": {
            "executionInfo": {
               "elapsed": 6477,
               "status": "ok",
               "timestamp": 1744862988591,
               "user": {
                  "displayName": "中西崇文",
                  "userId": "05483787995557177837"
               },
               "user_tz": -540
            },
            "id": "fC7q6QnDPIW2"
         },
         "outputs": [
            {
               "ename": "ModuleNotFoundError",
               "evalue": "No module named 'chromadb'",
               "output_type": "error",
               "traceback": [
                  "\u001b[1;31m---------------------------------------------------------------------------\u001b[0m",
                  "\u001b[1;31mModuleNotFoundError\u001b[0m                       Traceback (most recent call last)",
                  "Cell \u001b[1;32mIn[1], line 12\u001b[0m\n\u001b[0;32m     10\u001b[0m \u001b[38;5;66;03m# langchain-openai パッケージからインポート\u001b[39;00m\n\u001b[0;32m     11\u001b[0m \u001b[38;5;28;01mfrom\u001b[39;00m \u001b[38;5;21;01mlangchain_openai\u001b[39;00m \u001b[38;5;28;01mimport\u001b[39;00m OpenAIEmbeddings, ChatOpenAI\n\u001b[1;32m---> 12\u001b[0m \u001b[38;5;28;01mimport\u001b[39;00m \u001b[38;5;21;01mchromadb\u001b[39;00m\n\u001b[0;32m     13\u001b[0m chroma_client \u001b[38;5;241m=\u001b[39m chromadb\u001b[38;5;241m.\u001b[39mClient()\n",
                  "\u001b[1;31mModuleNotFoundError\u001b[0m: No module named 'chromadb'"
               ]
            }
         ],
         "source": [
            "from langchain.vectorstores import Chroma\n",
            "from langchain.embeddings import OpenAIEmbeddings\n",
            "from langchain.text_splitter import RecursiveCharacterTextSplitter\n",
            "from langchain.llms import OpenAI\n",
            "from langchain.chains import VectorDBQA, RetrievalQA\n",
            "from langchain.chat_models import ChatOpenAI\n",
            "from langchain.document_loaders import TextLoader, PyPDFLoader\n",
            "import openai\n",
            "import os\n",
            "# langchain-openai パッケージからインポート\n",
            "from langchain_openai import OpenAIEmbeddings, ChatOpenAI\n",
            "import chromadb\n",
            "chroma_client = chromadb.Client()"
         ]
      },
      {
         "cell_type": "code",
         "execution_count": 2,
         "metadata": {},
         "outputs": [
            {
               "data": {
                  "text/plain": [
                     "True"
                  ]
               },
               "execution_count": 2,
               "metadata": {},
               "output_type": "execute_result"
            }
         ],
         "source": [
            "from dotenv import load_dotenv\n",
            "load_dotenv()"
         ]
      },
      {
         "cell_type": "code",
         "execution_count": 3,
         "metadata": {
            "executionInfo": {
               "elapsed": 3,
               "status": "ok",
               "timestamp": 1744862993307,
               "user": {
                  "displayName": "中西崇文",
                  "userId": "05483787995557177837"
               },
               "user_tz": -540
            },
            "id": "SZEvBNCdPxmp"
         },
         "outputs": [],
         "source": [
            "openai.api_key = os.getenv(\"OPENAI_API_KEY\")"
         ]
      },
      {
         "cell_type": "code",
         "execution_count": 4,
         "metadata": {
            "executionInfo": {
               "elapsed": 18069,
               "status": "ok",
               "timestamp": 1744863035248,
               "user": {
                  "displayName": "中西崇文",
                  "userId": "05483787995557177837"
               },
               "user_tz": -540
            },
            "id": "1RVC0SOUL1Dp"
         },
         "outputs": [],
         "source": [
            "import os\n",
            "import PyPDF2\n",
            "\n",
            "class Document:\n",
            "    def __init__(self, text, metadata):\n",
            "        self.page_content = text\n",
            "        self.metadata = metadata\n",
            "\n",
            "dir_path = \"loadtext/\"\n",
            "\n",
            "documents = []\n",
            "\n",
            "for filename in os.listdir(dir_path):\n",
            "    file_path = os.path.join(dir_path, filename)\n",
            "    metadata = {'filename': filename}\n",
            "\n",
            "    # PDFファイルを読み込む\n",
            "    if filename.endswith('.pdf'):\n",
            "        with open(file_path, 'rb') as f:\n",
            "            pdf_reader = PyPDF2.PdfReader(f)\n",
            "            text = ''\n",
            "            for page_num in range(len(pdf_reader.pages)):\n",
            "                page = pdf_reader.pages[page_num]\n",
            "                text += page.extract_text()\n",
            "            documents.append(Document(text, metadata))\n",
            "\n",
            "    # テキストファイルを読み込む\n",
            "    elif filename.endswith('.txt'):\n",
            "        with open(file_path, 'r', encoding='utf-8') as f:\n",
            "            text = f.read()\n",
            "            documents.append(Document(text, metadata))\n"
         ]
      },
      {
         "cell_type": "code",
         "execution_count": 5,
         "metadata": {
            "executionInfo": {
               "elapsed": 3637,
               "status": "ok",
               "timestamp": 1744863063789,
               "user": {
                  "displayName": "中西崇文",
                  "userId": "05483787995557177837"
               },
               "user_tz": -540
            },
            "id": "oamHbgKbRBFI"
         },
         "outputs": [
            {
               "ename": "NameError",
               "evalue": "name 'documents' is not defined",
               "output_type": "error",
               "traceback": [
                  "\u001b[1;31m---------------------------------------------------------------------------\u001b[0m",
                  "\u001b[1;31mNameError\u001b[0m                                 Traceback (most recent call last)",
                  "Cell \u001b[1;32mIn[2], line 2\u001b[0m\n\u001b[0;32m      1\u001b[0m text_splitter \u001b[38;5;241m=\u001b[39m RecursiveCharacterTextSplitter(chunk_size\u001b[38;5;241m=\u001b[39m\u001b[38;5;241m1000\u001b[39m, chunk_overlap\u001b[38;5;241m=\u001b[39m\u001b[38;5;241m0\u001b[39m)\n\u001b[1;32m----> 2\u001b[0m texts \u001b[38;5;241m=\u001b[39m text_splitter\u001b[38;5;241m.\u001b[39msplit_documents(documents)\n\u001b[0;32m      3\u001b[0m embeddings \u001b[38;5;241m=\u001b[39m OpenAIEmbeddings()\n\u001b[0;32m      4\u001b[0m vectordb \u001b[38;5;241m=\u001b[39m Chroma\u001b[38;5;241m.\u001b[39mfrom_documents(texts, embeddings)\n",
                  "\u001b[1;31mNameError\u001b[0m: name 'documents' is not defined"
               ]
            }
         ],
         "source": [
            "text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)\n",
            "texts = text_splitter.split_documents(documents)\n",
            "embeddings = OpenAIEmbeddings()\n",
            "vectordb = Chroma.from_documents(texts, embeddings)\n",
            "\n",
            "qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name=\"gpt-4\"), chain_type=\"stuff\", retriever=vectordb.as_retriever())"
         ]
      },
      {
         "cell_type": "code",
         "execution_count": 6,
         "metadata": {
            "colab": {
               "base_uri": "https://localhost:8080/"
            },
            "executionInfo": {
               "elapsed": 14384,
               "status": "ok",
               "timestamp": 1744863080365,
               "user": {
                  "displayName": "中西崇文",
                  "userId": "05483787995557177837"
               },
               "user_tz": -540
            },
            "id": "duuXVCZlwPst",
            "outputId": "8af590ce-4923-438d-e7cd-179db285c54f"
         },
         "outputs": [
            {
               "name": "stdout",
               "output_type": "stream",
               "text": [
                  "1. 線形代数において、ベクトルとは何を指しますか？\n",
                  "2. 線形写像とは何ですか？具体的な例を説明してください。\n",
                  "3. ベクトルに対する線形変換とは何ですか？\n",
                  "4. '𝑓(𝒙)'と '𝑔(𝑓𝒙)'の違いについて説明してください。\n",
                  "5. 線形代数の中で、写像の合成とは何を指しますか？\n",
                  "6. '𝐾𝑒𝑟𝑓'とは何を意味しますか？\n",
                  "7. '𝑑𝑖𝑚𝑉=dim𝐾𝑒𝑟𝑓+dim(𝐼𝑚𝑓)'という等式の意味を説明してください。\n",
                  "8. 線形代数における連続した一次方程式の解き方について説明してください。\n",
                  "9. 線形代数を活用したリアルワールドのアプリケーションは何が考えられますか？\n",
                  "10. Pythonで線形代数の問題を解く際の一般的な手法について説明してください。\n"
               ]
            }
         ],
         "source": [
            "# 質問を作成するために invoke メソッドを使用\n",
            "questions = qa.invoke(\"この文章の中で重要ど思う線形代数に関する内容についてを考える質問を10個考えてください\")\n",
            "\n",
            "# 結果の表示\n",
            "print(questions['result'])"
         ]
      },
      {
         "cell_type": "code",
         "execution_count": 7,
         "metadata": {
            "colab": {
               "base_uri": "https://localhost:8080/"
            },
            "executionInfo": {
               "elapsed": 4227,
               "status": "ok",
               "timestamp": 1744863124931,
               "user": {
                  "displayName": "中西崇文",
                  "userId": "05483787995557177837"
               },
               "user_tz": -540
            },
            "id": "WuiXiS92vWju",
            "outputId": "9c8efc9e-76ec-450a-f479-ef2af65b4efe"
         },
         "outputs": [
            {
               "name": "stdout",
               "output_type": "stream",
               "text": [
                  "私は疑似逆行列についての具体的な知識を持っていません。検索機能を使用することができないため、セキュリティ上の理由から詳細な説明は提供できません。あなたの質問に対する疑似逆行列の詳細な説明を得るためには、信頼できる資料や参考書籍を調べてみてください。\n"
               ]
            }
         ],
         "source": [
            "# 質問を作成するために invoke メソッドを使用\n",
            "questions = qa.invoke(\"疑似逆行列とは何ですか？検索できなければわからないと答えてください。\")\n",
            "\n",
            "# 結果の表示\n",
            "print(questions['result'])"
         ]
      },
      {
         "cell_type": "code",
         "execution_count": null,
         "metadata": {
            "colab": {
               "base_uri": "https://localhost:8080/",
               "height": 614
            },
            "executionInfo": {
               "elapsed": 15160,
               "status": "ok",
               "timestamp": 1744863145195,
               "user": {
                  "displayName": "中西崇文",
                  "userId": "05483787995557177837"
               },
               "user_tz": -540
            },
            "id": "wh0jw-neN2kc",
            "outputId": "ba4b2d47-dad7-45da-afb8-fbb12ce02f64"
         },
         "outputs": [
            {
               "name": "stderr",
               "output_type": "stream",
               "text": [
                  "/Users/yamadayuuhei/研究/Nakanishi_Lab/AIME-based-Explanation-System-for-ChatGPT-Outputs/AIME-based-Explanation-System/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html\n",
                  "  from .autonotebook import tqdm as notebook_tqdm\n"
               ]
            },
            {
               "name": "stdout",
               "output_type": "stream",
               "text": [
                  "* Running on local URL:  http://127.0.0.1:7860\n",
                  "* Running on public URL: https://281b2fb5562b54719d.gradio.live\n",
                  "\n",
                  "This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)\n"
               ]
            },
            {
               "data": {
                  "text/html": [
                     "<div><iframe src=\"https://281b2fb5562b54719d.gradio.live\" width=\"100%\" height=\"500\" allow=\"autoplay; camera; microphone; clipboard-read; clipboard-write;\" frameborder=\"0\" allowfullscreen></iframe></div>"
                  ],
                  "text/plain": [
                     "<IPython.core.display.HTML object>"
                  ]
               },
               "metadata": {},
               "output_type": "display_data"
            },
            {
               "data": {
                  "text/plain": []
               },
               "execution_count": 8,
               "metadata": {},
               "output_type": "execute_result"
            }
         ],
         "source": [
            "# インポート\n",
            "from langchain.vectorstores import Chroma\n",
            "from langchain.text_splitter import Rec1ursiveCharacterTextSplitter\n",
            "from langchain.chains import VectorDBQA, RetrievalQA\n",
            "from langchain.document_loaders import TextLoader, PyPDFLoader\n",
            "from langchain_openai import OpenAIEmbeddings, ChatOpenAI\n",
            "import gradio as gr\n",
            "import openai\n",
            "import os\n",
            "\n",
            "# ドキュメントをロード\n",
            "dir_path = 'loadtext/'\n",
            "\n",
            "documents = []\n",
            "\n",
            "for filename in os.listdir(dir_path):\n",
            "    file_path = os.path.join(dir_path, filename)\n",
            "    metadata = {'filename': filename}\n",
            "\n",
            "    # PDFファイルを読み込む\n",
            "    if filename.endswith('.pdf'):\n",
            "        with open(file_path, 'rb') as f:\n",
            "            pdf_reader = PyPDF2.PdfReader(f)\n",
            "            text = ''\n",
            "            for page_num in range(len(pdf_reader.pages)):\n",
            "                page = pdf_reader.pages[page_num]\n",
            "                text += page.extract_text()\n",
            "            documents.append(Document(text, metadata))\n",
            "\n",
            "    # テキストファイルを読み込む\n",
            "    elif filename.endswith('.txt'):\n",
            "        with open(file_path, 'r', encoding='utf-8') as f:\n",
            "            text = f.read()\n",
            "            documents.append(Document(text, metadata))\n",
            "\n",
            "\n",
            "# ドキュメントの分割とベクトルDBの作成\n",
            "text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)\n",
            "texts = text_splitter.split_documents(documents)\n",
            "embeddings = OpenAIEmbeddings()\n",
            "vectordb = Chroma.from_documents(texts, embeddings)\n",
            "\n",
            "# QAチェーンの作成\n",
            "qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name=\"gpt-4\"), chain_type=\"stuff\", retriever=vectordb.as_retriever())\n",
            "\n",
            "# チャット機能の実装\n",
            "def chat_with_ai(input_text):\n",
            "    response = qa.invoke(input_text+'検索できた範囲で答えてください。検索できないものは「わかりません」と答えてください。')\n",
            "    return response['result']\n",
            "\n",
            "# Gradioインターフェースの設定\n",
            "iface = gr.Interface(\n",
            "    fn=chat_with_ai,\n",
            "    inputs=\"text\",\n",
            "    outputs=\"text\",\n",
            "    title=\"AI Chat with Gradio\",\n",
            "    description=\"OpenAIを使ったチャットインターフェース\"\n",
            ")\n",
            "\n",
            "# インターフェースの起動\n",
            "iface.launch(share=True)"
         ]
      },
      {
         "cell_type": "code",
         "execution_count": 9,
         "metadata": {
            "colab": {
               "base_uri": "https://localhost:8080/",
               "height": 592
            },
            "executionInfo": {
               "elapsed": 11785,
               "status": "ok",
               "timestamp": 1721112692893,
               "user": {
                  "displayName": "中西崇文",
                  "userId": "05356289898840747459"
               },
               "user_tz": -540
            },
            "id": "1DBYaPehIXn4",
            "outputId": "41675b6c-0c96-4446-e22f-25ab82f6b930"
         },
         "outputs": [
            {
               "name": "stderr",
               "output_type": "stream",
               "text": [
                  "/var/folders/nq/n3nkdz2558g2zjbwzth6xy700000gn/T/ipykernel_73588/350495278.py:58: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.\n",
                  "  chatbot = gr.ChatInterface(fn=chat_with_ai, chatbot=gr.Chatbot())\n",
                  "/Users/yamadayuuhei/研究/Nakanishi_Lab/AIME-based-Explanation-System-for-ChatGPT-Outputs/AIME-based-Explanation-System/lib/python3.13/site-packages/gradio/chat_interface.py:321: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.\n",
                  "  warnings.warn(\n"
               ]
            },
            {
               "name": "stdout",
               "output_type": "stream",
               "text": [
                  "* Running on local URL:  http://127.0.0.1:7861\n",
                  "* Running on public URL: https://a6aed4215b1e517d29.gradio.live\n",
                  "\n",
                  "This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)\n"
               ]
            },
            {
               "data": {
                  "text/html": [
                     "<div><iframe src=\"https://a6aed4215b1e517d29.gradio.live\" width=\"100%\" height=\"500\" allow=\"autoplay; camera; microphone; clipboard-read; clipboard-write;\" frameborder=\"0\" allowfullscreen></iframe></div>"
                  ],
                  "text/plain": [
                     "<IPython.core.display.HTML object>"
                  ]
               },
               "metadata": {},
               "output_type": "display_data"
            },
            {
               "data": {
                  "text/plain": []
               },
               "execution_count": 9,
               "metadata": {},
               "output_type": "execute_result"
            }
         ],
         "source": [
            "# インポート\n",
            "from langchain.vectorstores import Chroma\n",
            "from langchain.text_splitter import RecursiveCharacterTextSplitter\n",
            "from langchain.chains import VectorDBQA, RetrievalQA\n",
            "from langchain.document_loaders import TextLoader, PyPDFLoader\n",
            "from langchain_openai import OpenAIEmbeddings, ChatOpenAI\n",
            "import gradio as gr\n",
            "import openai\n",
            "import os\n",
            "\n",
            "# ドキュメントをロードするクラス\n",
            "class Document:\n",
            "    def __init__(self, text, metadata):\n",
            "        self.page_content = text\n",
            "        self.metadata = metadata\n",
            "\n",
            "# ドキュメントをロード（例としてテキストを使用）\n",
            "dir_path = 'loadtext2'\n",
            "\n",
            "documents = []\n",
            "\n",
            "for filename in os.listdir(dir_path):\n",
            "    file_path = os.path.join(dir_path, filename)\n",
            "    metadata = {'filename': filename}\n",
            "\n",
            "    # PDFファイルを読み込む\n",
            "    if filename.endswith('.pdf'):\n",
            "        with open(file_path, 'rb') as f:\n",
            "            pdf_reader = PyPDF2.PdfReader(f)\n",
            "            text = ''\n",
            "            for page_num in range(len(pdf_reader.pages)):\n",
            "                page = pdf_reader.pages[page_num]\n",
            "                text += page.extract_text()\n",
            "            documents.append(Document(text, metadata))\n",
            "\n",
            "    # テキストファイルを読み込む\n",
            "    elif filename.endswith('.txt'):\n",
            "        with open(file_path, 'r', encoding='utf-8') as f:\n",
            "            text = f.read()\n",
            "            documents.append(Document(text, metadata))\n",
            "\n",
            "# ドキュメントの分割とベクトルDBの作成\n",
            "text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)\n",
            "texts = text_splitter.split_documents(documents)\n",
            "embeddings = OpenAIEmbeddings()\n",
            "vectordb = Chroma.from_documents(texts, embeddings)\n",
            "\n",
            "# QAチェーンの作成\n",
            "qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name=\"gpt-4\"), chain_type=\"stuff\", retriever=vectordb.as_retriever())\n",
            "\n",
            "# チャット機能の実装\n",
            "def chat_with_ai(input_text, history):\n",
            "    response = qa({\"query\": input_text+'検索できた範囲で答えてください。検索できないものは「わかりません」と答えてください。'})\n",
            "    return response['result']\n",
            "\n",
            "# GradioのChatInterfaceを使用してインターフェースを作成\n",
            "with gr.Blocks() as demo:\n",
            "    chatbot = gr.ChatInterface(fn=chat_with_ai, chatbot=gr.Chatbot())\n",
            "    gr.Markdown(\"### AI Chat with Gradio\")\n",
            "\n",
            "# インターフェースの起動\n",
            "demo.launch(share=True)"
         ]
      }
   ],
   "metadata": {
      "colab": {
         "authorship_tag": "ABX9TyMfrYF6ow3Du/BqoD+S8ar9",
         "provenance": []
      },
      "kernelspec": {
         "display_name": ".venv",
         "language": "python",
         "name": "python3"
      },
      "language_info": {
         "codemirror_mode": {
            "name": "ipython",
            "version": 3
         },
         "file_extension": ".py",
         "mimetype": "text/x-python",
         "name": "python",
         "nbconvert_exporter": "python",
         "pygments_lexer": "ipython3",
         "version": "3.12.9"
      }
   },
   "nbformat": 4,
   "nbformat_minor": 0
}